# Create_Clusters

In [ ]:
import rasterio
from rasterio.mask import mask
from rasterio.features import shapes, rasterize
from rasterstats import zonal_stats
import geopandas as gpd
import json
from pathlib import Path

from rasterio.plot import show
import numpy as np
from scipy import ndimage
import time

start_time = time.time()

# 1. Download

GHS layer https://ghsl.jrc.ec.europa.eu/data.php
admin GADM
grid from Alex

In [ ]:
# Input GHS (global) GeoTiff, output clipped and shapefile to use for clipping
folder_input = Path('/home/chris/Documents/GIS')
ghs_in = folder_input / 'GHS-POP/GHS_POP_GPW42015_GLOBE_R2015A_54009_250_v1_0.tif'

clip_boundary = folder_input / 'gadm_uganda.gpkg'
clip_boundary_layer = 'gadm36_UGA_0'

grid_in = folder_input / 'uganda_grid.gpkg'

# 2. Clip

https://automating-gis-processes.github.io/CSC18/lessons/L6/clipping-raster.html

In [ ]:
ghs = rasterio.open(str(ghs_in))

# open shapefile and convert to string of points for clipping bounds
adm = gpd.read_file(str(clip_boundary), layer=clip_boundary_layer, driver='GPKG')
adm = adm.to_crs(crs=ghs.crs)
coords = [json.loads(adm.to_json())['features'][0]['geometry']]

# mask/clip the raster using rasterio.mask
ghs_clipped, ghs_clipped_transform = mask(dataset=ghs, shapes=coords, crop=True)

# 3. Vectorize

https://gis.stackexchange.com/questions/187877/how-to-polygonize-raster-to-shapely-polygons

In [ ]:
ghs_geoms = list(({'properties': {'raster_val': v}, 'geometry': s} 
              for i, (s, v)
              in enumerate(shapes(ghs_clipped, mask=None, transform=ghs_clipped_transform))))

ghs_poly = gpd.GeoDataFrame.from_features(ghs_geoms)
ghs_poly.crs = ghs.crs.data

# 4. Filter on population and size, buffer and dissolve

In [ ]:
max_block_size_multi = 5  # remove blocks bigger than x times average
min_block_pop = 50
buffer_amount = 150

# filter to ignore blocks with basically no people
ghs_poly['area_m2'] = ghs_poly.geometry.area
ghs_poly = ghs_poly[ghs_poly['area_m2'] < ghs_poly['area_m2'].mean() * max_block_size_multi] # remove blocks that are too big (basically artifacts)
ghs_poly = ghs_poly[ghs_poly['raster_val'] > min_block_pop] # remove blocks with 30 or less people

# buffer outwards so that nearby blocks will overlap
ghs_poly['geometry'] = ghs_poly.geometry.buffer(buffer_amount)

# and dissolve the thousands of blocks into a single shapefile (with no attributes!)
ghs_poly['same'] = 1
ghs_poly = ghs_poly.dissolve(by='same')

# 7. To singleparts

https://gis.stackexchange.com/a/271735

In [ ]:
# To get our attributes, we convert the dissolves polygon into singleparts
# This means each contiguous bubble becomes its own polygon and can store its own attributes

ghs_poly = ghs_poly.explode()
ghs_poly = ghs_poly.reset_index()
ghs_poly['geometry'] = ghs_poly[0]
ghs_poly = ghs_poly.drop(columns=['level_0', 'level_1', 0]) # shapefile doesn't like integer column name
ghs_poly = gpd.GeoDataFrame(ghs_poly)
ghs_poly.crs = ghs.crs.data

# 8. Raster zonal statistics

https://automating-gis-processes.github.io/CSC18/lessons/L6/zonal-statistics.html

In [ ]:
# But we still need to get the population data back, so we join it with the original raster data
# We take the sum of all population that lies underneath the polygon
pop_sums = zonal_stats(ghs_poly, ghs_in, stats='sum')
ghs_poly['pop_sum'] = [x['sum'] for x in pop_sums]

# And then add the polygon's area back to its attributes
ghs_poly["area_m2"] = ghs_poly['geometry'].area

# 9. Get grid distances from raster

In [ ]:
# read in the relevant MV grid lines file
grid = gpd.read_file(grid_in)
grid = grid.to_crs(crs=ghs_poly.crs)

grid_raster = rasterize(grid.geometry, out_shape=ghs_clipped[0].shape, fill=1,
                        default_value=0, all_touched=True, transform=ghs_clipped_transform)
dist_raster = ndimage.distance_transform_edt(grid_raster) * 0.25

dists = zonal_stats(vectors=ghs_poly, raster=dist_raster, affine=ghs_clipped_transform, stats='min', nodata=1000)
ghs_poly['grid_dist'] = [x['min'] for x in dists]

# 11. Save output files

In [ ]:
ghs_poly = ghs_poly.to_crs(epsg=4326)
ghs_poly.to_file('clusters_processed.gpkg', driver='GPKG')

print(time.time() - start_time)